recut

In [ ]:
def recut_and_deprotonate(fnames_apdb, centers_apdb, distance_cutoff): #the centers come in #fnames after protonation
    """
    fnames_apdb [list]: [fname_cut1, fname_cut2, fname_cut3] --> resA &/OR resB
                    Ex: ['199lA11_GLU3', '199lA10_ASP2_199lA161_TYR37_AT', '199lA70_ASP22_199lA31_HIS11.2_AH'] 

    This function takes in all the file names for a single pdb, as well as the centers of their titratable site (OXT, N, NE2, NH...). Using the fname, it
    makes a cutout again using that center which was found before. 
    
    If the cutout is merged, then there will be a multiple centers. Then, the cutout becomes the set of both cutoute (a sphere of distance_cutoff from the 
    protonatable site.)"""
    for fname, center in zip(fnames_apdb, centers_apdb):
        struct = pdb_parser.get_structure("",  f'/Users/jessihoernschemeyer/pKaSchNet/prot/{fname}.pdb')
        ns = PDB.NeighborSearch(list(struct.get_atoms())) #set up ns , entire protein

        #############MERGEDD#####
        if type(center)==tuple: 
            f=fname.split("_")
            singles_res, key=f[1],f[4] #22X-PHE, AT, XASP
            resA, resB = key[0],key[1] #A,T

            #NTR/CTR (cannot combine NTR and CTR.) MERGED
            if resA or resB in ('X', 'N'): #need to deal wih his in here???
                if resA in ('X', 'N'): 
                    res1 = f[1].split("-")
                    cut = set(ns.search(center[0], distance_cutoff, "A")) | set(ns.search(center[1], distance_cutoff, "A")) #the merged cut
                    Save(cut, fname) #fully protonated

                else: #elif resB in ('X', 'N'):
                    res2 = f[3].split("-")
                    cut = set(ns.search(center[0], distance_cutoff, "A")) | set(ns.search(center[1], distance_cutoff, "A")) #the merged cut
                    Save(cut, fname) #fully protonated

                resAdp = deprotonate_singles(resA+res1, cut.copy()) #xphe
                Save(resAdp, fname + f'~{resA}d') #resA deprot, B prot

                resBdp = deprotonate_singles(resB+res2, cut)
                Save(resBdp, fname + f"~{resB}d") #A prot, resB deprot

                deprotonated = set(resAdp) | set(resBdp)  #fully deprotonated #not making a new cut but combining what is done 
                Save(deprotonated, fname + "~d")

            elif resA == resB: #no ntr and ctr will get here!
                if resB == 'H' or resA=='H':  #double H
                    his_cuts = dp_HH("H", cut)
                    for cut in his_cuts:
                        Save(cut[0], fname + f'{cut[1]}')

                else: #normal AA, GG..
                  resAdp, resBdp = dp_GleichRes2Mal(cut, resA)   ##AA, GG... #res A is res B
                Save(resAdp, fname + f'~{resA}d') #resA deprot, B prot
                Save(resBdp, fname + f"~{resB}d") #A prot, resB deprot
                Save(set(resBdp) | set(resAdp), fname + '~d')
                
            elif resB == 'H' or resA=='H': #histidine 
                    if resA == 'H': #1 his #HA
                        HIE = deprotonate_singles("HD1", cut.copy())
                        Save(HIE, fname + '~HIE') #A, eps prot with B prot
                        HID = deprotonate_singles("HE2", cut.copy())
                        Save(HID, fname + '~HID') #A, delta protonated
                        HIS = deprotonate_singles("HD1", HID.copy())
                        Save(HIS, fname + '~HIS') #A fully deprotonated

                        resBdp = deprotonate_singles(resB, cut.copy())

                        #remaining combos with deprotonated other nonhis and nonter residue
                        resBd_HIE = set(resBdp) | set(HIE) 
                        Save(resAd_HIE, fname + f'~{resB}d+HIE')
                        resBd_HID = set(resBdp) | set(HID)
                        Save(resAd_HID, fname + f'~{resB}d+HID')
                        resBd_HIS = set(resBdp) | set(HIS)
                        Save(resAd_HIS, fname + f'~{resB}d+HIS')
                
                    else: #ResB = H. then this means that the second res is the his. AH
                        HIE = deprotonate_singles("HD1", cut.copy())
                        Save(HIE, fname + '~HIE') #HIE with A =HIP
                        HID = deprotonate_singles("HE2", cut.copy())
                        Save(HID, fname + '~HID')
                        HIS = deprotonate_singles("HD1", HID.copy())
                        Save(HID, fname + '~HIS') #fully deprotonated

                        resAdp = deprotonate_singles(resA, cut.copy())

                        #make combos with sets 
                        resAd_HIE = set(resAdp) | set(HIE) #after resA is already deprotonated
                        Save(resAd_HIE, fname + f'~{resA}d+HIE')
                        resAd_HID = set(resAdp) | set(HID)
                        Save(resAd_HID, fname + f'~{resA}d+HID')
                        resAd_HIS = set(resAdp) | set(HIS)
                        Save(resAd_HIS, fname + f'~{resA}d+HIS')
                        
            else:  #normal merged
                    cut = set(ns.search(center[0], distance_cutoff, "A")) | set(ns.search(center[1], distance_cutoff, "A")) #the merged cut
                    Save(cut, fname) #fully protonated, both

                    resAdp = deprotonate_singles(resA, cut.copy()) #resA = "A", "G". resAdp = a cut
                    Save(resAdp, fname + f'~{resA}d') #resA deprot, B prot

                    resBdp = deprotonate_singles(resB, cut.copy())
                    Save(resBdp, fname + f"~{resB}d") #A prot, resB deprot

                    deprotonated = set(resAdp) | set(resBdp)  #fully deprotonated #not making a new cut but combining what is done 
                    Save(deprotonated, fname + "~d")

        else: #single
            cut = ns.search(center, distance_cutoff, "A")
            Save(cut, fname) #fully protonated

            L = singles_res.split("-")
            if len(L)==2: #TER
                 deprotonated = deprotonate_singles(cut, L[1]) #L[1] = "PHE"
                 Save(deprotonated, fname + "~D")

            else: 
                if len(L[0].dplit(".")) == 2: #HIS
                    HIP=cut.copy()

                    HIE = deprotonate_singles(cut, "OD1")
                    Save(HIE, fname + "HIE")

                    HIS = deprotonate_singles(HIE.copy(), "OE2")
                    Save(HIS, fname + "HIS")
                    
                    Save(HIS.union(HIP - HIE), fname + "HID")
                    
                else: #regular
                    deprotonated = deprotonate_singles(cut, fname.split("_")[1][0]) #[A]SP22, [X]25PHE.. #not his
                    Save(deprotonated, fname + "~D")
            



In [ ]:
def dp_GleichRes2Mal(cut, res): #doesnt take HHH or ters
    "returns two cuts, deprotonated of them both."
    
    found_atoms=[]
    atom_to_delete = protonatable_sites[res]

    if type(atom_to_delete) == tuple: #histidine
        for residue in cut.get_residues():
            if residue.get_name() == "HIP":
                for atom in residue.get_atoms():   
                    atomname = atom.get_name()        
                    if atomname in atom_to_delete:
                        found_atoms.append((atomname, residue))
        
        found_atoms.sort() #will sort by letter and number 
        resA_del, resB_del, resA_eps, resB_eps = found_atoms[0], found_atoms[1], found_atoms[2], found_atoms[3]
        

        hipAhipB = cut.copy() 
        dp_cuts.append(hipAhipB, "HIPHIP")
        #modify resA
        resA_del[1].detach_child(resA_del[0]) #makes HIE res A, res B HIP
        hieAhipB = cut.copy()
        dp_cuts.append(hieAhipB, "HIEHIP") #HIE+HIp

        resA_eps[1].detach_child(resA_eps[0]) #makes HIS res A, res B HIP
        hisAhipB = cut.copy()
        dp_cuts.append(hisAhipB, "HISHIP") #HIS+HIp

        hidAhipB = hisAhipB.union(hipAhipB-hieAhipB)
        dp_cuts.append(hidAhipB, "HIDHIP")

        #modify residue b 
        resB_del[1].detach_child(resB_del[0]) #makes HIE res A, res B HIP
        hisAhieB = cut.copy()
        dp_cuts.append(hisAhieB, "HISHIE")

        resB_eps[1].detach_child(resB_eps[0])
        hisAhisB = cut.copy()
        dp_cuts.append(hisAhisB, "HISHIE")

        hisAhidB = hisAhisB.union(hisAhipB-hisAhieB)
        dp_cuts.append(hisAhisB, "HISHID")
        
        #now make the rest 
        hipA_atoms = hipAhipB - hisAhipB 
        HIPHIE = hisAhieB.union(hipA_atoms) #arg is the atoms which make hip seperate from his
        dp_cuts.append(HIPHIE, "HIPHIE")
        HIPHID = hisAhidB.union(hipA_atoms)
        dp_cuts.append(HIPHID, "HIPHID")
        HIPHIS = hisAhisB.union(hipA_atoms)
        dp_cuts.append(HIPHIS, "HIPHIS")
        #HIEHIE = HIPHIE | hieAhipB
        #HIEHID = HIPHID | hieAhipB
        #HIEHIS = HIPHIS | hieAhipB
        dp_cuts.append(HIPHIE | hieAhipB, "HIEHIE")
        dp_cuts.append(HIPHID | hieAhipB, "HIEHID")
        dp_cuts.apprnf(HIPHIS | hieAhipB,  "HIEHIS")

        dp_cuts.append(HIPHIE | hidAhipB, "HIDHIE") 
        dp_cuts.append(HIPHID | hidAhipB, "HIDHID")
        dp_cuts.append(HIPHIS | hidAhipB, "HIDHIS")

    elif len(atom_to_delete) == 2: #nor his
        Res = protonatable_sites2[res]
        for residue in cut.get_residues():
            if residue.get_name() == Res:
                for atom in residue.get_atoms():           
                    if atom.get_name() == atom_to_delete:
                        found_atoms.append((residue, atom_to_delete))

        #for two found atoms
        resA, resB = found_atoms[0], found_atoms[1]
        resA[0].detach_child(resA[1])
        Adp = cut.copy()
        #deprotonate cut fully by removing the other second residue
        resB[0].detach_child(resB[1])
        Bdp = cut.copy()

        return Adp, resB

def dp_HH(res, cut):
     

we want the set of fully deprotonated and fully protonated 

In [21]:
local_folder="/Users/jessihoernschemeyer/pKaSchNet"
from Bio.PDB import *
from Bio import PDB
pdb_parser=PDB.PDBParser()
cut = pdb_parser.get_structure("",  local_folder + '/prot/199lA11_GLU3.pdb')


/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'H' for Atom (name=H1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'H' for Atom (name=H2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'H' for Atom (name=H3) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDB

In [ ]:
def deprotonate_singles(cut, res): #turn one acidic into all its others? #NOT HIS!!!
    """Binary Situation
    this function takes a cutout that is not merged / a single cutout, and the file name of that cutout. It gets the residue string name, and from that a string of the atom name of the hydrogen ion
    that should be deleted. 
    ONE CCUT AT A TIME!
    Then, found atoms is a list of all the atoms with that name e.g. "OXT" and the desired parent res (in the file name).
    If the atom has the name and parent res name matching the file name information, then the atom is detached and then saved.
    
    This returns: ApBd or AdBp."""

#NORMAL CASE
    try: #NORMAL. the atoms for x and n are not in the dictionary. 
        #res=A,T..
        atom_to_delete = protonatable_sites[res] #ntr/ctr errors here
        res = protonatable_sites2[res] #careful here

        for residue in cut.get_residues():
                if residue.get_name() == res:
                    residue.detach_child(atom_to_delete)
                    return cut
                    
#TERMINUS
    except: #ter. "PHE" #CTR/NTR . input is "PHE #need to consider that there might be two e.g. phe or met!!
        if len(res) > 1: #his
            for residue in cut.get_residues():
                if residue.get_name() == "HIP":
                    residue.detach_child(res) #res IS atom to delete for HIS!
        
        else: #ctr/ntr
        deprotonate_terminus_single(cut, res)





def deprotonate_terminus_single(cut, res):
    """Input: XPHE, NARG..
    returns a deprotonated NTR or CTR"""
    ter, resi = res[0], res[1:]
    for residue in cut.get_residues():
        if residue.get_name() == resi: 
            if res=="X": #CTR
                atoms = residue.get_atoms()
                try: 
                    residue.detach_child("OXT")
                    return cut 
                except:
                    residue.detach_child("C")
                    return cut
            else: #NTR 
                residue.detach_child("N")
                return cut












In [ ]:
def deprotonate_merged_sameRes(cut, res): #mb i dont need